<a href="https://colab.research.google.com/github/GenericP3rson/Pokemon-Notebooks/blob/main/extended_pokemon_lab_create_graph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![pokemon](https://people.emich.edu/jhe5/fall16COSC231/project1/banner.jpg)
This lab is based on Shreya Chaudhary's Medium blog on [**Using API Data with TigerGraph**](https://medium.com/@shreya_chaudhary/ef98cc9293d3)

Checkout Akash Kaul's blog on [**How to Get Started With TigerGraph**](https://medium.com/@akash_kaul/4aac0ca4fb3d) for some nice visualizations of the same processed outlined below.


# Getting Started Building Your Own Pokemon Graph


## 0.0 Data Sneak Peak

In [ ]:
# Locations

import requests
import json

In [ ]:
URL = "https://pokeapi.co/api/v2/pokemon/12/encounters" # URL
res = requests.get(url = URL).json() # Makes the GET Request
print(json.dumps(res, indent = 2)) # Printing the Results

In [ ]:
# print(res["species"])
print(json.dumps(res[0]["location_area"], indent = 2))

{
  "name": "kanto-route-2-south-towards-viridian-city",
  "url": "https://pokeapi.co/api/v2/location-area/296/"
}


In [ ]:
URL = f"https://pokeapi.co/api/v2/region/1/" # URL
res = requests.get(url = URL).json() # We don't have any parameters.

print(res["name"])
print(res["locations"][0]["name"])

kanto
celadon-city


## 1.0 Setting Up Your TigerGraph Box
To get started with this lab, the first thing you must have is a TigerGraph instance that you can store all your data. Don't worry it's much easier than your thinking. The next few steps will walk you through the process. 

### 1.1 Create an Account
> <font color='red'>If you have already created an account simply sign in and skip the create account step.</font>


* Click on this link to go to TigerGraph's Cloud [https://tgcloud.io](https://tgcloud.io). You will be prompted with a "Login" button. Click the botton. 
* Next you will see a top tab where you can choose "Sign Up". Click that tab.
* Finally you should see an option to create an account using your Google Account, LinkedIn Account, or manually create an account. Choose the option you perfer. 

### 1.2 Create a TigerGraph Instance
* After creating an account and getting into your portal you should see some options on the left side. One of those options is "My Soultions". Click "My Solutions"
* On the top right corner of your screen you will see "Create Solution" Click the "Creat Solutions button.

> <font color='red'>There are four steps we will be going through to create the box</font>

#### 1.2.1 Basic Settings
  * Select TigerGraph Version: `3.0.5`
  * Select a Starter Kit: `Blank v3`
  * Click `"Next"` at the bottom

#### 1.2.2 Instance Settings
* Select a Platform: `AWS`
* Select an Instance Type: `TG.FREE`
* Select Region: `(any - choose the closest to you)`
* Disk Size: `50GB`
* Partition Factor: `1`
* Replication Factor: `1`
* Click `"Next"` at the bottom

#### 1.2.3 Solution Settings
* Name your Solution: `(any - choose a name you would like)`
* Tag your Solution: `(any - choose a tag you would like)`
* Set the Initial Password: `tigergraph`
* Subdomain: `(any - choose a name you would like)`
* Description: `(any - choose a description you would like)`
* Click `"Next"` at the bottom

#### 1.2.4 Confirmation
* Make sure everything looks good
* Click `"Submit"` if things look good.

### 1.3 CONGRATULATIONS!
Let's take a look at your new box. It might take ~3-5 minutes for the box to be ready. You can see the box being provisioned if you head back to "My Solutions" tab.

#### 1.3.1 Open Graph Studio
Once the box is showing a <font color='green'>green</font> dot, you can click on the first icon next to that dot called "Applications". From the list choose "GraphStudio". This is the Graphical User Interface that communicates with your graph. 

<font color='red'> Let's move on to building the graph database using your Google Colab Notebook along with some Python tools. </font>

## 2.0 Creating Pokemon Graph

### 2.1 Grab Tools to Guild the Graph
Below you will see a few things we will be installing along with importing into our Python enviorment. 
* pyTigerGraph is a python package used to communicate to TigerGraph
* Import will grab those tools to then use in our lab
  *  We import request for APIs, JSON for file formatting, and pyTigerGraph as the TigerGraph connector.

In [ ]:
!pip install pyTigerGraph
import requests
import json
import pyTigerGraph as tg

### 2.2 Setup Connection Details

In [ ]:
conn = tg.TigerGraphConnection(host="https://shreya.i.tgcloud.io", password="tigergraph", gsqlVersion="3.0.5", useCert=True)

### 2.3 Create Schema Elements for our Edges and Vertices

First, we're going to need to create a schema. This is **how** the data will be presented. What sorts of connections should we have? What exactly should we load?

As shown below, we have vertices and edges. Vertices are similar to an item and the edges shows connections between the items. PRIMARY_ID is the unique name of the vertex. Anything after that are attributes; they provide more details about the vertex.

In [ ]:
# Sets up connection with GSQL
print(conn.gsql('ls', options=[]))
# Create Edges (POKEMON_TYPE) and Vertices (Pokemon and Type)
print(conn.gsql('''
CREATE VERTEX Pokemon (PRIMARY_ID name STRING, image STRING, weight INT, height INT) WITH primary_id_as_attribute="true"
CREATE VERTEX Type (PRIMARY_ID type_id STRING) WITH primary_id_as_attribute="true"
CREATE VERTEX Gender (PRIMARY_ID gender STRING) WITH primary_id_as_attribute="true"
CREATE VERTEX Item (PRIMARY_ID item STRING, rarity INT) WITH primary_id_as_attribute="true"
CREATE VERTEX Ability (PRIMARY_ID ability STRING) WITH primary_id_as_attribute="true"
CREATE VERTEX Move (PRIMARY_ID move STRING, accuracy INT, effect_chamce INT, power_powers INT, priority INT, power INT) WITH primary_id_as_attribute="true"
CREATE VERTEX LocationArea (PRIMARY_ID location_area STRING) WITH primary_id_as_attribute="true"
CREATE VERTEX Location (PRIMARY_ID location STRING) WITH primary_id_as_attribute="true"

CREATE UNDIRECTED EDGE LOCATION_AREA_LOCATION (FROM LocationArea, TO Location)
CREATE UNDIRECTED EDGE POKEMON_LOCATION_AREA (FROM Pokemon, TO LocationArea)
CREATE UNDIRECTED EDGE POKEMON_MOVE (FROM Pokemon, TO Move)
CREATE UNDIRECTED EDGE POKEMON_GENDER (FROM Pokemon, TO Gender)
CREATE UNDIRECTED EDGE POKEMON_ITEM (FROM Pokemon, TO Item)
CREATE UNDIRECTED EDGE POKEMON_ABILITY (FROM Pokemon, TO Ability)
CREATE UNDIRECTED EDGE POKEMON_TYPE (FROM Pokemon, TO Type)
''', options=[]))


Trying version: v3_0_5
Connecting to shreya.i.tgcloud.io:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
---- Global vertices, edges, and all graphs
Vertex Types: 
Edge Types: 

Graphs: 
Jobs: 


JSON API version: v2
Syntax version: v1


Trying version: v3_0_5
Connecting to shreya.i.tgcloud.io:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
The vertex type Pokemon is created.
The vertex type Type is created.
The vertex type Gender is created.
The vertex type Item is created.
The vertex type Ability is created.
The vertex type Move is created.
The vertex type LocationArea is created.
The vertex type Location is created.
The edge type LOCATION_AREA_LOCATION is created.
The edge type POKEMON_LOCATION_AREA is created.
The edge type POKEMON_MOVE is created.
The edge type POKEMON_GENDER is created.
The edge type POKEMON_ITEM is created.
The edge type POKEMON_ABILITY is created.
The edge type POKEMON_TYPE is created.



You just created the schema! Congrats! Let's walk through what exactly you loaded.

### Vertices
- Pokemon
  - This is a Pokémon itself!
  - Attributes: 
    - name: Name of the Pokémon 
    - image: Image URL
    - weight: weight
    - height: height
- Type 
  - This is the type of the Pokémon!
  - Attributes:
    - type_id: type (fire, air, etc.) 
- Gender
  - This is the gender of the Pokémon.
  - Attributes:
    - gender: gender of Pokémon (m/f)
- Item 
  - This is an item the Pokémon might be holding.
  - Attributes:
    - item: the name of the item
    - rarity: how rare the item is
- Ability
  - An sbility that the Pokémon has
  - Attributes:
    - ability: the name of the ability
- Move 
  - A move the Pokémon can do!
  - Attributes:
    - move: the name of the move
    - accuracy: how accurate the move is
    - effect_chance: the probability that the move will occur 
    - power_powers: amount of points the move is worth
    - priority: prioirty of the move
    - power: power of the move
- LocationArea 
  - This is the region the Pokémon is in
  - Attributes:
    - location_area: name of the region
- Location 
  - This is the more specific place the Pokémon is in
  - Attributes:
    - location: name of the place

### Edges
The next part are the edges. They are fairly straightforward, connecting one vertex to another.

They include:
- LOCATION_AREA_LOCATION
- POKEMON_LOCATION_AREA
- POKEMON_MOVE
- POKEMON_GENDER
- POKEMON_ITEM
- POKEMON_ABILITY
- POKEMON_TYPE

### 2.4 Create Pokemon Graph

In [ ]:
print(conn.gsql('''CREATE GRAPH PokemonGraph(Pokemon, Type, Gender, Item, Ability, Move, Location, LocationArea, LOCATION_AREA_LOCATION, POKEMON_LOCATION_AREA, POKEMON_MOVE, POKEMON_GENDER, POKEMON_ITEM, POKEMON_ABILITY, POKEMON_TYPE)''', options=[])) # Create the Graph

Trying version: v3_0_5
Connecting to shreya.i.tgcloud.io:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
Stopping GPE GSE RESTPP
Successfully stopped GPE GSE RESTPP in 0.005 seconds
Starting GPE GSE RESTPP
Successfully started GPE GSE RESTPP in 0.104 seconds
The graph PokemonGraph is created.



### 2.5 Update our Connection Parameters

In [ ]:
conn.graphname = "PokemonGraph"
conn.apiToken = conn.getToken(conn.createSecret())

## 3.0 Loading Data into Pokemon Graph

In [ ]:
### Let's start off by looking at our API data

URL = "https://pokeapi.co/api/v2/pokemon/12/" # URL
res = requests.get(url = URL).json() # Makes the GET Request
print(json.dumps(res, indent = 2)) # Printing the Results

{
  "abilities": [
    {
      "ability": {
        "name": "compound-eyes",
        "url": "https://pokeapi.co/api/v2/ability/14/"
      },
      "is_hidden": false,
      "slot": 1
    },
    {
      "ability": {
        "name": "tinted-lens",
        "url": "https://pokeapi.co/api/v2/ability/110/"
      },
      "is_hidden": true,
      "slot": 3
    }
  ],
  "base_experience": 178,
  "forms": [
    {
      "name": "butterfree",
      "url": "https://pokeapi.co/api/v2/pokemon-form/12/"
    }
  ],
  "game_indices": [
    {
      "game_index": 125,
      "version": {
        "name": "red",
        "url": "https://pokeapi.co/api/v2/version/1/"
      }
    },
    {
      "game_index": 125,
      "version": {
        "name": "blue",
        "url": "https://pokeapi.co/api/v2/version/2/"
      }
    },
    {
      "game_index": 125,
      "version": {
        "name": "yellow",
        "url": "https://pokeapi.co/api/v2/version/3/"
      }
    },
    {
      "game_index": 12,
      "version"

In [ ]:
### We can now upsert or add one Pokémon and its species

conn.upsertVertex("Pokemon", res["species"]["name"], attributes={"name": res["species"]["name"] })

1

In [ ]:
### Here, we upsert the different types of Pokémon and make the connections as necessary

for ty in res["types"]:
     conn.upsertVertex("Type", ty["type"]["name"], attributes={"type_id": ty["type"]["name"] })
     conn.upsertEdge("Pokemon", res["species"]["name"], "POKEMON_TYPE", "Type", ty["type"]["name"])

In [ ]:
### We now Grab Location Information; we'll load this into the graph
for i in range(1, 9):
    URL = f"https://pokeapi.co/api/v2/region/{i}/" # URL
    res = requests.get(url = URL).json() # We don't have any parameters.
    conn.upsertVertex("LocationArea", res["name"], attributes={"location_area": res["name"]})
    # for ty in res["locations"]:
    #     conn.upsertVertex("Location", ty["name"], attributes={"location": ty["name"]})
    #     conn.upsertEdge("LocationArea", res["name"], "LOCATION_AREA_LOCATION", "Location", ty["name"])
    print("Added " + res["name"])

Added kanto
Added johto
Added hoenn
Added sinnoh
Added unova
Added kalos
Added alola
Added galar


In [ ]:

### Finally, we'll add all the Pokémon and the vertices connected to them.

for i in range(1,100): # You can change the number based on how many Pokémon you want to add
    URL = f"https://pokeapi.co/api/v2/pokemon/{i}/" # URL
    res = requests.get(url = URL).json() # We don't have any parameters.
    conn.upsertVertex("Pokemon", res["species"]["name"], attributes={"name": res["species"]["name"], "image":  res["sprites"]["other"]["official-artwork"]["front_default"], "weight": res["weight"], "height": res["height"]})
    for ty in res["types"]:
        conn.upsertVertex("Type", ty["type"]["name"], attributes={"type_id": ty["type"]["name"] })
        conn.upsertEdge("Pokemon", res["species"]["name"], "POKEMON_TYPE", "Type", ty["type"]["name"])
    for ty in res["abilities"]: # Loads abilities
        conn.upsertVertex("Ability", ty["ability"]["name"], attributes={"ability": ty["ability"]["name"] })
        conn.upsertEdge("Pokemon", res["species"]["name"], "POKEMON_ABILITY", "Ability", ty["ability"]["name"])
    # for ty in res["held_items"]:
    #     conn.upsertVertex("Item", ty["item"]["name"], attributes={"item": ty["item"]["name"], "rarity": ty["version_details"][0]["rarity"] })
    #     conn.upsertEdge("Pokemon", res["species"]["name"], "POKEMON_ITEM", "Item", ty["item"]["name"])
    for ty in res["moves"]:
        conn.upsertVertex("Move", ty["move"]["name"], attributes={"move": ty["move"]["name"] })
        conn.upsertEdge("Pokemon", res["species"]["name"], "POKEMON_MOVE", "Move", ty["move"]["name"])
    
    print("Added " + res["species"]["name"])

# Location, Region

Added bulbasaur
Added ivysaur
Added venusaur
Added charmander
Added charmeleon
Added charizard
Added squirtle
Added wartortle
Added blastoise
Added caterpie
Added metapod
Added butterfree
Added weedle
Added kakuna
Added beedrill
Added pidgey
Added pidgeotto
Added pidgeot
Added rattata
Added raticate
Added spearow
Added fearow
Added ekans
Added arbok
Added pikachu
Added raichu
Added sandshrew
Added sandslash
Added nidoran-f
Added nidorina
Added nidoqueen
Added nidoran-m
Added nidorino
Added nidoking
Added clefairy
Added clefable
Added vulpix
Added ninetales
Added jigglypuff
Added wigglytuff
Added zubat
Added golbat
Added oddish
Added gloom
Added vileplume
Added paras
Added parasect
Added venonat
Added venomoth
Added diglett
Added dugtrio
Added meowth
Added persian
Added psyduck
Added golduck
Added mankey
Added primeape
Added growlithe
Added arcanine
Added poliwag
Added poliwhirl
Added poliwrath
Added abra
Added kadabra
Added alakazam
Added machop
Added machoke
Added machamp
Added bellsp

In [ ]:
URL = "https://pokeapi.co/api/v2/pokemon/12/encounters" # URL
res = requests.get(url = URL).json() # Makes the GET Request
# print(json.dumps(res, indent = 2)) # Printing the Results

# print(json.dumps(res[0]["location_area"]["name"].split("-")[0], indent = 2))

for i in range(1, 20):
    URL = f"https://pokeapi.co/api/v2/pokemon/{i}/" # URL
    res = requests.get(url = URL).json() # We don't have any parameters.
    pokemon_name = res["species"]["name"] # Pokemon Name
    URL = f"https://pokeapi.co/api/v2/pokemon/{i}/encounters" # URL
    res = requests.get(url = URL).json() # Makes the GET Request
    for loc in res:
        conn.upsertEdge("Pokemon", pokemon_name, "POKEMON_LOCATION_AREA", "LocationArea", loc["location_area"]["name"].split("-")[0])

Action Item
---
* [ ] Load Attack / Defense (HP?) Stats 
* [ ] Query to get attack/defense data + pokemon + type
* [ ] Create Linear Regression
* [ ] Types correspond to colours!!!

In [ ]:
import plotly.express as px
import pandas as pd

# Let's start basic with the weight and height first

result = conn.runInstalledQuery("allPokemon", {})[0]["catchThemAll"]
height = []
weight = []
types = []
'''
Things to Do:
- Types
- Names
- Images??????????

'''
for i in result:
  print(i["attributes"])
  height.append(i["attributes"]["height"])
  weight.append(i["attributes"]["weight"])

d = {'height': height, 'weight': weight}
df = pd.DataFrame(data=d)
# print(result[0]["catchThemAll"])

# df = px.data.tips()
fig = px.scatter(df, x="height", y="weight", trendline="ols")
fig.show()

NameError: ignored

In [ ]:
# UN-COMMENT this if you want to drop all your data
# print(conn.gsql('''DROP ALL''', options=[]))

Trying version: v3_0_5
Connecting to shreya.i.tgcloud.io:14240
If there is any relative path, it is relative to <System.AppRoot>/dev/gdk/gsql
Dropping all, about 1 minute ...
Abort all active loading jobs
Try to abort all loading jobs on graph PokemonGraph, it may take a while ...
[ABORT_SUCCESS] No active Loading Job to abort.
Resetting GPE...
Successfully reset GPE
Stopping GPE GSE
Successfully stopped GPE GSE in 6.667 seconds
Clearing graph store...
Successfully cleared graph store
Everything is dropped.

